# Segmenting and Clustering Neighborhoods in Toronto

## Table of Contents

1.Download and Explore Dataset

2.Explore Neighborhoods in Toronto

3.Analyze Each Neighborhood

4.Cluster Neighborhoods 

5.Examine Clusters

Import libararies that will be used for the analysis.

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup
from urllib.request import urlopen

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset

Scrape the Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the list of postal codes of Toronto. 

In [6]:
# parse the wikipedia page and transform the data into a dataframe df

html = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()

soup = BeautifulSoup(html, features='lxml')

My_table = soup.find('table',{'class':'wikitable sortable'})

data = []

for record in My_table.findAll('td'):
    data.append(record.text)

PostalCode = data[0::3]
Borough = data[1::3]
Neighborhood = data[2::3]
Neighborhood = [line.rstrip('\n') for line in Neighborhood]

df = pd.DataFrame()
df['PostalCode'] = PostalCode
df['Borough'] = Borough
df['Neighborhood'] = Neighborhood

df = df[df.Borough != 'Not assigned']

df = df.groupby(['PostalCode','Borough'], as_index=False, sort=False).agg(','.join)

# for Borough "Queen's Park", replace its Neighborhood column value with "Queen's Park"

df.replace('Not assigned', 'Queen\'s Park', inplace = True)

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [7]:
# read the csv file that contains the geographical coordinates of each postal code in Toronto

geo = pd.read_csv('geo.csv')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
# rename the column "Postal Code" to "PostalCode"
geo.rename(columns = {'Postal Code': 'PostalCode'}, inplace = True)

# merge the new dataframe toronto with the previous dataframe df

MetroToronto = pd.merge(df, geo, on = "PostalCode")

MetroToronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(MetroToronto['Borough'].unique()),
        MetroToronto.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


Since we are only interested in boroughs that contain the word Toronto, we will slice the original dataframe and create a new dataframe of the Toronto data. 

In [12]:
trt = ['Downtown Toronto', 'Central Toronto', 'West Toronto', 'East Toronto']
toronto = MetroToronto[MetroToronto.Borough.isin(trt)].reset_index(drop=True)

toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Use geopy library to get the latitude and longitude values of Toronto.

In [14]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Visualize the neighborhoods in Toronto.

In [15]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## 2. Explore Neighborhoods in Toronto

Utilize the Foursquare API to explore the neighborhoods and segment them.

In [16]:
CLIENT_ID = 'JGZNWOCWTNWGTNXRJPB3CZWBRYXARMEXVI4PY2WVMY4NEVOX' # your Foursquare ID
CLIENT_SECRET = 'J0IFJCEHRZYBRHKUWC2ZIC45R0AP3I0LRNCSW1VNRXAWUDRV' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JGZNWOCWTNWGTNXRJPB3CZWBRYXARMEXVI4PY2WVMY4NEVOX
CLIENT_SECRET:J0IFJCEHRZYBRHKUWC2ZIC45R0AP3I0LRNCSW1VNRXAWUDRV


Create a function to get nearby venues of all the neighborhoods in Toronto.

In [19]:
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=2500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each neighborhood and create a new dataframe called "toronto_venues".

In [20]:
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city


Check the size of the resulting dataframe.

In [21]:
print(toronto_venues.shape)

(3800, 7)


There were 3800 vanues returned for all of the neighborhoods. Let's check how many venues were returned for each neighborhood.

In [22]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"Brockton,Exhibition Place,Parkdale Village",100,100,100,100,100,100
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",100,100,100,100,100,100
"Cabbagetown,St. James Town",100,100,100,100,100,100
Central Bay Street,100,100,100,100,100,100
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,100,100,100,100,100,100


There were 100 venues returned for each neighborhood. Let's find out how many unique categories were curated. 

In [25]:
toronto_venues['Venue Category'].unique()

array(['Bakery', 'Coffee Shop', 'Historic Site', 'Farmers Market',
       'Mediterranean Restaurant', 'Chocolate Shop', 'Restaurant',
       'Gym / Fitness Center', 'Dessert Shop', 'Park',
       'Italian Restaurant', 'Performing Arts Venue', 'Liquor Store',
       'Thai Restaurant', 'French Restaurant', 'Café', 'Pool', 'Hotel',
       'Diner', 'Middle Eastern Restaurant', 'BBQ Joint',
       'Athletics & Sports', 'German Restaurant', 'Bistro',
       'Breakfast Spot', 'Grocery Store', 'Gastropub', 'Food Truck',
       'Seafood Restaurant', 'Japanese Restaurant', 'Curling Ice',
       'Creperie', 'Taco Place', 'Bar', 'Circus', 'Cosmetics Shop',
       'Theater', 'Pub', 'Farm', 'Steakhouse', 'Garden',
       'American Restaurant', 'Church', 'Beer Bar', 'Burrito Place',
       'Cocktail Bar', 'Museum', 'Ramen Restaurant', 'Gym',
       'Clothing Store', 'Pizza Place', 'Ice Cream Shop', 'Jewelry Store',
       'Indian Restaurant', 'Shopping Mall', 'Vietnamese Restaurant',
       'Comic Sh

In [24]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 221 uniques categories.


## 3. Analyze Each Neighborhood

In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Castle,Cemetery,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Nudist Beach,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [29]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Zoo,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Castle,Cemetery,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nudist Beach,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.0,0.00,0.01,0.0,0.0,0.01,0.02,0.0,0.01,0.00,0.00,0.01,0.0,0.0,0.00,0.0,0.00,0.0,0.02,0.01,0.00,0.03,0.0,0.00,0.01,0.00,0.01,0.01,0.01,0.0,0.0,0.00,0.0,0.00,0.02,0.0,0.04,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.01,0.00,0.09,0.01,0.00,0.01,0.02,0.0,0.01,0.00,0.01,0.00,0.0,0.00,0.01,0.0,0.0,0.01,0.02,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.02,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.01,0.01,0.0,0.0,0.0,0.00,0.0,0.00,0.02,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.03,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.05,0.00,0.00,0.00,0.00,0.0,0.0,0.01,0.03,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.01,0.00,0.00,0.01,0.00,0.01,0.01,0.01,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.01,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.03,0.0,0.02,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.01,0.02,0.0,0.01,0.00,0.02,0.01,0.01,0.01,0.01,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.01,0.01,0.02,0.00,0.0,0.02,0.0,0.00,0.

Print each neighborhood along with the top 5 most common venues

In [31]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
         venue  freq
0  Coffee Shop  0.09
1        Hotel  0.05
2         Café  0.04
3      Theater  0.04
4          Gym  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1                Café  0.05
2               Hotel  0.05
3  Italian Restaurant  0.04
4      Cosmetics Shop  0.03


----Brockton,Exhibition Place,Parkdale Village----
          venue  freq
0          Café  0.08
1    Restaurant  0.05
2           Bar  0.05
3   Coffee Shop  0.04
4  Cocktail Bar  0.03


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0         Café  0.09
1  Coffee Shop  0.07
2         Park  0.06
3      Brewery  0.05
4        Beach  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                venue  freq
0                Park  0.07
1         Coffee Shop  0.04
2  Italian Restaurant  0.04
3          Restaurant  0.04
4              Bakery  0.03


-

Write a function to sort the venues in descending order.

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood.

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Hotel,Café,Theater,Pizza Place,Gym,Japanese Restaurant,Beer Bar,Steakhouse,Sporting Goods Shop
1,Berczy Park,Coffee Shop,Hotel,Café,Italian Restaurant,Theater,Farmers Market,Cosmetics Shop,Park,Japanese Restaurant,Steakhouse
2,"Brockton,Exhibition Place,Parkdale Village",Café,Bar,Restaurant,Coffee Shop,Italian Restaurant,Asian Restaurant,Athletics & Sports,Bakery,Cocktail Bar,Theater
3,Business Reply Mail Processing Centre 969 Eastern,Café,Coffee Shop,Park,Brewery,Beach,Bakery,Diner,American Restaurant,Italian Restaurant,Bar
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Park,Italian Restaurant,Restaurant,Coffee Shop,Bakery,Café,Hotel,Yoga Studio,Music Venue,Spa


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 3 clusters.

In [40]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:40] 

array([0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 2, 2, 2, 0, 1, 0, 2, 1, 0, 0, 1,
       2, 1, 2, 2, 1, 2, 2, 1, 0, 0, 0, 1, 2, 1, 1, 1], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [41]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Café,Park,Diner,Restaurant,Japanese Restaurant,Gastropub,Farmers Market,Italian Restaurant,Cocktail Bar
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Hotel,Japanese Restaurant,Café,Theater,Thai Restaurant,Farmers Market,Gastropub,Italian Restaurant,Steakhouse
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Hotel,Thai Restaurant,Theater,Farmers Market,Japanese Restaurant,Plaza,Italian Restaurant,Burrito Place
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Beach,Pub,Café,Coffee Shop,Park,Bakery,Breakfast Spot,Thai Restaurant,Bar,Ice Cream Shop
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Hotel,Café,Italian Restaurant,Theater,Farmers Market,Cosmetics Shop,Park,Japanese Restaurant,Steakhouse


Visualize the resulting clusters.

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Examine each cluster and determine the discriminating venue categories that distinguish each cluster.

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Café,Park,Diner,Restaurant,Japanese Restaurant,Gastropub,Farmers Market,Italian Restaurant,Cocktail Bar
1,Downtown Toronto,0,Coffee Shop,Hotel,Japanese Restaurant,Café,Theater,Thai Restaurant,Farmers Market,Gastropub,Italian Restaurant,Steakhouse
2,Downtown Toronto,0,Coffee Shop,Café,Hotel,Thai Restaurant,Theater,Farmers Market,Japanese Restaurant,Plaza,Italian Restaurant,Burrito Place
4,Downtown Toronto,0,Coffee Shop,Hotel,Café,Italian Restaurant,Theater,Farmers Market,Cosmetics Shop,Park,Japanese Restaurant,Steakhouse
5,Downtown Toronto,0,Café,Coffee Shop,Pizza Place,Concert Hall,Restaurant,Vegetarian / Vegan Restaurant,Hotel,Theater,Steakhouse,Park
7,Downtown Toronto,0,Coffee Shop,Hotel,Café,Theater,Pizza Place,Gym,Japanese Restaurant,Beer Bar,Steakhouse,Sporting Goods Shop
9,Downtown Toronto,0,Coffee Shop,Hotel,Café,Theater,Steakhouse,Gym,Park,French Restaurant,Sporting Goods Shop,Movie Theater
12,Downtown Toronto,0,Hotel,Coffee Shop,Café,Theater,Park,Beer Bar,Baseball Stadium,Pizza Place,Steakhouse,Plaza
15,Downtown Toronto,0,Café,Hotel,Coffee Shop,Theater,Thai Restaurant,Japanese Restaurant,Farmers Market,Baseball Stadium,Concert Hall,Steakhouse
31,Downtown Toronto,0,Park,Italian Restaurant,Restaurant,Coffee Shop,Bakery,Café,Hotel,Yoga Studio,Music Venue,Spa


In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,1,Beach,Pub,Café,Coffee Shop,Park,Bakery,Breakfast Spot,Thai Restaurant,Bar,Ice Cream Shop
6,Downtown Toronto,1,Café,Korean Restaurant,Coffee Shop,Bar,Vegetarian / Vegan Restaurant,Pizza Place,Ice Cream Shop,Indian Restaurant,Beer Bar,Park
8,West Toronto,1,Café,Bar,Italian Restaurant,Coffee Shop,Park,Bakery,Brewery,Cocktail Bar,Dessert Shop,Pub
10,West Toronto,1,Café,Bar,Cocktail Bar,Bakery,Italian Restaurant,Pizza Place,Coffee Shop,Asian Restaurant,Yoga Studio,Bookstore
11,East Toronto,1,Greek Restaurant,Café,Park,Vietnamese Restaurant,Bakery,Italian Restaurant,Pub,American Restaurant,Pizza Place,Gastropub
13,West Toronto,1,Café,Bar,Restaurant,Coffee Shop,Italian Restaurant,Asian Restaurant,Athletics & Sports,Bakery,Cocktail Bar,Theater
14,East Toronto,1,Coffee Shop,Café,Park,Beach,Brewery,Gastropub,Ice Cream Shop,Bakery,BBQ Joint,Bar
16,East Toronto,1,Coffee Shop,Park,Café,Vietnamese Restaurant,Brewery,Bar,Thai Restaurant,Pizza Place,Diner,French Restaurant
21,West Toronto,1,Café,Coffee Shop,Bar,Italian Restaurant,Bakery,Pizza Place,Park,Brewery,Gastropub,Restaurant
24,West Toronto,1,Coffee Shop,Café,Bar,Bakery,Park,Restaurant,Italian Restaurant,Gastropub,Sushi Restaurant,Eastern European Restaurant


In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,2,Coffee Shop,Bakery,Sushi Restaurant,Italian Restaurant,Café,Tea Room,Park,Japanese Restaurant,Yoga Studio,Pizza Place
18,Central Toronto,2,Coffee Shop,Italian Restaurant,Café,Japanese Restaurant,Bakery,Park,Middle Eastern Restaurant,Trail,Gym,Deli / Bodega
19,Central Toronto,2,Coffee Shop,Café,Italian Restaurant,Bakery,Park,Sushi Restaurant,Gastropub,Gym,Indian Restaurant,Sporting Goods Shop
20,Central Toronto,2,Café,Coffee Shop,Italian Restaurant,Dessert Shop,Park,Gastropub,Middle Eastern Restaurant,Sushi Restaurant,Bakery,Spa
22,Central Toronto,2,Coffee Shop,Café,Italian Restaurant,Bakery,Sushi Restaurant,Park,Japanese Restaurant,Gastropub,Burger Joint,Tea Room
23,Central Toronto,2,Café,Coffee Shop,Italian Restaurant,Park,Spa,Pizza Place,Ice Cream Shop,Grocery Store,Korean Restaurant,Japanese Restaurant
25,Central Toronto,2,Café,Coffee Shop,Italian Restaurant,Gym,Sushi Restaurant,Bakery,Indian Restaurant,Park,Gastropub,Spa
28,Central Toronto,2,Italian Restaurant,Park,Café,Indian Restaurant,Grocery Store,Hotel,Dessert Shop,Coffee Shop,Spa,Yoga Studio
30,Central Toronto,2,Italian Restaurant,Café,Park,Coffee Shop,Spa,BBQ Joint,Ice Cream Shop,French Restaurant,Vegetarian / Vegan Restaurant,Middle Eastern Restaurant
32,Downtown Toronto,2,Italian Restaurant,Coffee Shop,Park,Spa,Gastropub,Restaurant,Japanese Restaurant,French Restaurant,Concert Hall,Men's Store
